In [1]:
import pandas as pd
import os
import numpy as np

import plotly.express as px

from data.constants import DATA_FOLDER

from power import power_uncond
from tabulate import  tabulate

In [2]:
comm_panel_in = os.path.join(DATA_FOLDER, "final", "comm_panel.parquet")

In [3]:
comm_panel = pd.read_parquet(comm_panel_in)
comm_panel = comm_panel.query('~is_weekend')

In [4]:
trt_panel = comm_panel.query("(DNC==1)")

In [5]:
power_stats_base = power_uncond(trt_panel, ['transit','UCMP'], 'rides', 0)[['transit','UCMP','mean','std','mde','mds']]
power_stats_base['mds'] /= 5e4
plot_data = power_stats_base.round(1).set_index(['UCMP','transit']).sort_index()

In [6]:
plot_data.index.set_names(['Serves DNC', 'Transit'], inplace=True)
plot_data.columns.set_names('Daily Rides', inplace=True)
plot_data['Mean (SD)'] = plot_data['mean'].astype(str) + plot_data['std'].apply(" ({})".format)
plot_data.drop(columns=['mean','std'], inplace=True)
plot_data.rename(columns={'mde': 'Minimum Detectable Effect',
                          'mds': 'MDE Per Attendee'}, inplace=True)
plot_data = plot_data.iloc[:, [2,0,1]]
plot_data

Daily Rides                 Mean (SD)  Minimum Detectable Effect  \
Serves DNC Transit                                                 
0          bike        424.8 (1271.9)                      223.6   
           train     6732.4 (11439.2)                     2708.5   
           uber       4525.4 (8956.9)                     1521.5   
1          bike       1289.3 (1609.8)                      797.3   
           train      7417.2 (5733.2)                     3035.4   
           uber     10842.0 (13352.4)                     6612.8   

Daily Rides         MDE Per Attendee  
Serves DNC Transit                    
0          bike                  1.1  
           train                 7.6  
           uber                  8.3  
1          bike                  0.5  
           train                 1.7  
           uber                  4.2

In [7]:
with open("../../../reports/replication/mde.md","w") as f:
    tbl = tabulate(plot_data.reset_index(), 
                   headers=plot_data.reset_index().columns, 
                   showindex=False,
                   tablefmt='github')
    f.write(tbl)
    print(tbl)
!cp ../../../reports/replication/mde.md ../../../../eric-mc2-cv/static/uploads/mde.md

|   Serves DNC | Transit   | Mean (SD)         |   Minimum Detectable Effect |   MDE Per Attendee |
|--------------|-----------|-------------------|-----------------------------|--------------------|
|            0 | bike      | 424.8 (1271.9)    |                       223.6 |                1.1 |
|            0 | train     | 6732.4 (11439.2)  |                      2708.5 |                7.6 |
|            0 | uber      | 4525.4 (8956.9)   |                      1521.5 |                8.3 |
|            1 | bike      | 1289.3 (1609.8)   |                       797.3 |                0.5 |
|            1 | train     | 7417.2 (5733.2)   |                      3035.4 |                1.7 |
|            1 | uber      | 10842.0 (13352.4) |                      6612.8 |                4.2 |
